In [5]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [6]:
class ThresholdReLU(nn.Module):
    def __init__(self, threshold=0.001):
        super().__init__()
        self.threshold = threshold
    def forward(self, x):
        return torch.where(x > self.threshold, x, torch.zeros_like(x))

def quantize_input(x, levels=16, lo=0.0, hi=1.0):
    x = x.clamp(lo, hi)
    x_norm = (x - lo) / (hi - lo)
    x_q = torch.round(x_norm * (levels - 1)) / (levels - 1)
    return x_q * (hi - lo) + lo

def sparsity_loss_modified(activations, beta=20.0):
    total = sum(a.numel() for a in activations)
    loss = 0.0
    for a in activations:
        loss += torch.sum(1.0 - torch.tanh(beta * a.abs()))
    return loss / total


In [7]:
class VGG16_Sparse(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # load standard VGG16-BN feature extractor
        self.features = torchvision.models.vgg16_bn(pretrained=False).features
        # replace the classifier for CIFAR‑10
        self.classifier = nn.Sequential(
            nn.Linear(512*1*1, 4096),
            ThresholdReLU(),
            nn.Linear(4096, 4096),
            ThresholdReLU(),
            nn.Linear(4096, num_classes)
        )
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.relu = ThresholdReLU()

    def forward(self, x):
        acts = []
        for layer in self.features:
            x = layer(x)
            # record activations immediately after each BatchNorm
            if isinstance(layer, nn.BatchNorm2d):
                acts.append(x.clone())
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x, acts

In [8]:
def generate_sparsity_adversary(model, x_clean, y_clean, criterion,
                                epsilon=0.5, alpha=0.05, num_iter=120,
                                c=0.0, beta=50.0):
    model.eval()
    x_adv = x_clean.detach().clone().to(device)
    x_adv.requires_grad = True

    for i in range(num_iter):
        logits, acts = model(x_adv)
        l_sp = sparsity_loss_modified(acts, beta=beta)
        loss = l_sp  # c=0 → ignore CE
        model.zero_grad()
        loss.backward()
        # gradient step
        x_adv = x_adv - alpha * x_adv.grad.sign()
        # project into L∞ ball around x_clean
        x_adv = torch.max(torch.min(x_adv, x_clean+epsilon),
                          x_clean-epsilon).clamp(0.0, 1.0)
        x_adv = x_adv.detach()
        x_adv.requires_grad = True

        if (i+1) % 30 == 0:
            print(f"WB iter {i+1}/{num_iter}, SP loss: {l_sp.item():.4f}")

    return x_adv.detach()

In [9]:
class SPSAAttack:
    def __init__(self, model, bounds=(0,1), sigma=2e-3, lr=1e-2,
                 max_iter=500, targeted=True, samples=4):
        self.model     = model
        self.lo, self.hi = bounds
        self.sigma     = sigma
        self.lr        = lr
        self.max_iter  = max_iter
        self.targeted  = targeted
        self.samples   = samples

    def attack(self, x_orig, tgt_label):
        delta = torch.zeros_like(x_orig, device=x_orig.device)
        tgt   = torch.tensor([tgt_label], device=x_orig.device)

        for _ in range(self.max_iter):
            grad_est = torch.zeros_like(delta)
            for _ in range(self.samples):
                u = torch.randint(0,2,x_orig.shape,device=x_orig.device).float()*2 - 1
                x_p = torch.clamp(x_orig + delta + self.sigma*u, self.lo, self.hi)
                x_n = torch.clamp(x_orig + delta - self.sigma*u, self.lo, self.hi)
                lp,_ = self.model(x_p); ln,_ = self.model(x_n)
                l_p = F.cross_entropy(lp, tgt)
                l_n = F.cross_entropy(ln, tgt)
                diff = (l_p - l_n) if self.targeted else (l_n - l_p)
                grad_est += diff * u / (2*self.sigma)
            grad_est /= self.samples
            delta = delta - self.lr * grad_est
            delta = torch.clamp(delta, self.lo - x_orig, self.hi - x_orig)
        return torch.clamp(x_orig + delta, self.lo, self.hi).detach()

In [10]:
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std),
])
testset  = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# ─── 7) Instantiate & Load ────────────────────────────
model     = VGG16_Sparse().to(device)
model.eval()
surrogate = model
target    = copy.deepcopy(model)
surrogate.eval(); target.eval()

criterion = nn.CrossEntropyLoss()
spsa = SPSAAttack(model=target)

Files already downloaded and verified


/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
# ─ 8) Two‑Stage Black‑Box Sparsity Attack ──
for imgs, _ in testloader:
    imgs = imgs.to(device)
    # (1) Clean preds & activations
    with torch.no_grad():
        logits_c, acts_c = target(quantize_input(imgs))
        preds_c = logits_c.argmax(1)

    # (2) White‑box stage on surrogate
    imgs_q  = quantize_input(imgs)
    imgs_s1 = generate_sparsity_adversary(
                   surrogate, imgs_q, preds_c, criterion,
                   epsilon=0.5, alpha=0.05, num_iter=120, c=0.0, beta=50.0
               )

    # (3) SPSA repair any flips
    imgs_adv = imgs_s1.clone()
    for i in range(imgs.size(0)):
        with torch.no_grad():
            pi = target(imgs_s1[i:i+1])[0].argmax(1)
        if pi != preds_c[i]:
            imgs_adv[i:i+1] = spsa.attack(imgs_s1[i:i+1], preds_c[i].item())

    # (4) Measure sparsity
    with torch.no_grad():
        _, acts_a = target(quantize_input(imgs_adv))
    sp_c = sum((a!=0).float().sum() for a in acts_c)/sum(a.numel() for a in acts_c)
    sp_a = sum((a!=0).float().sum() for a in acts_a)/sum(a.numel() for a in acts_a)

    print(f"Sparsity clean: {sp_c:.4f}, adversarial: {sp_a:.4f}")
    break

WB iter 30/120, SP loss: 0.0562
WB iter 60/120, SP loss: 0.0544
WB iter 90/120, SP loss: 0.0536
WB iter 120/120, SP loss: 0.0531
Sparsity clean: 0.9358, adversarial: 1.0000
